In [1]:
import os
import csv
import numpy as np
import pandas as pd
import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pylab import plot,show,hist
from scipy.stats.kde import gaussian_kde
from scipy.stats import norm, chi2_contingency
import statsmodels.api as sm
from numpy import linspace,hstack
from pylab import plot,show,hist
import pydot
#%config InlineBackend.figure_format = 'svg' для большей четкости графиков
matplotlib.style.use('ggplot')
%matplotlib inline

#Стандартизация данных
from sklearn import preprocessing

#Для построения диаграмм рассеивания
from pandas.plotting import scatter_matrix

#Графика для интерпретации моделей
from IPython.display import Image
from sklearn.tree import export_graphviz
from subprocess import call

#Факторный анализ
import math
from sklearn import preprocessing
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import PCA

#SVD - Singular Value decomposition
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate


os.chdir(r'C:\Users\Mr Alex\Documents\GitHub\FlightPreparence')
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num',
           'marital-status', 'occupation', 'relationship', 'race', 'sex',
           'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']


#df = pd.read_csv('AmesHousing.txt', sep="\t", header = 0, index_col=False)
#df = pd.read_csv('town_1959_2.csv', header = 0,)
#df = pd.read_csv('swiss_bank_notes.csv', index_col=0)
#df = pd.read_csv('beverage_r.csv', sep=";", index_col='numb.obs')
#df = pd.read_csv('Protein Consumption in Europe.csv', sep=';', decimal=',', index_col='Country')
#df = pd.read_csv('assess.dat', sep='\t', index_col='NAME')
#df = pd.read_csv('Albuquerque Home Prices_data.txt', sep='\t')
#df = pd.read_csv('agedeath.dat.txt', sep='\s+', header=None, names=['group', 'age', 'index'])
#df = pd.read_csv('interference.csv')
#df = pd.read_csv('diamond.dat', header=None, sep='\s+', names=['weight', 'price'])
#df = pd.read_csv('Credit.csv', sep=';', encoding='cp1251')
#df = pd.read_csv('adult.test', header=None, names=columns, na_values=' ?', skiprows=1)
#df = pd.read_csv('Wine.txt', sep='\t', header=0)
#df = pd.read_csv('monthly-car-sales-in-quebec-1960.csv', sep=';', header=0, parse_dates=[0])
#df = pd.read_csv('stickleback.csv', sep=';', decimal=',')
df = pd.read_csv('Swiss Fertility.csv', sep=';', decimal=',', index_col=0)

In [2]:
def scale_features(df):
    """
    Функция для стандартизации переменных в датафрейме
    """
    scaled = preprocessing.StandardScaler().fit_transform(df)
    scaled = pd.DataFrame(scaled, columns=df.columns)
    return scaled

In [ ]:
#Факторный анализ 
#Сокращение переменных через введение новых, искусственных переменных (факторов), которые их заменяют
#Способы поиска наилучших проекций: projection persuit, многомерное шкалирование,карты Sommer
#Выявление структур взаимозависимости данных, матриц корреляции
#Преодоление мультиколинеарности переменных в регрессионом анализе
#R - матрица корелляции=k*k, где k - число столбцов исходной матрицы. Она же дисперсия вектора 
#U - (уникальности) особенности в данных, которые не удается объеснить факторами. Определяет качество модели 

#Удаляем столбцы, где данные непрочитались
df = df.filter(regex='^(?!.*Unnamed).*$')

#Смотрим коэффициенты корелляций. Мало больших значений - плохо для факторного анализа
df.corr()

#Строим R матрицу корелляций. Много выбросов, есть бимодальности. Но сильной корелляции увы нет
scatter_matrix(df); #Добавление ";" позволяет показать только график, без цифр

#Cтандартизируем переменные
df_scaled = preprocessing.scale(df)

#Методом поиска главных компонентов проецируем данные на двумерную плоскость и получаем ранжирование компонентов по важности 
pca = PCA(n_components=5).fit(df_scaled) #Уточняем число компонент и источник данных 

#Доля разброса в данных, объясняемая главными компонентами
print('Влияние компонентов на общий разброс данных: ', pca.explained_variance_ratio_)
#Чистые значения главных компонент
meanings = pca.singular_values_

#Массив, в котором посчитаны значения факторов, заменяющие исходный набор данных
pca_factor = pca.transform(df_scaled)

#Запускаем факторный анализ. Сравним влияние факторов
fa = FactorAnalysis(n_components=2).fit(df_scaled) #Факторов задаем много, сокращаем пока не получим адекватную группировку

#Таблица коэффициентов корреляции - что именно измерили факторы. Семь параметров превращаются в два новых фактора
pd.DataFrame(fa.components_, columns=df.columns) #Обновление данных, где те же колонки, но переменные в них - факториалы

#Дисперсия остатков U. Видно, что хотя новые факторы очевидны, остатки огромные, то есть переменные объяснены плохо
#Если некоторые переменные объясняются очень плохо, то они уникальны и преображать их в факторы не надо
pd.Series(fa.noise_variance_, df.columns) 

#Значения факторов можно применить к конкретным объектам
scores = pd.DataFrame(fa.transform(df_scaled), columns=['factor1', 'factor2'])
scores['factor1'].sort_values() #Сортируем по значению фактора

In [ ]:
#SVD-разложение Simon Funk'a для рекомендательных систем. Позволяет работать с данными, где много пропусков 
#Двойной факторный анализ, проведенный одновременно. Матрица триплетов получается из: номер стр, номер стлб., элемент.
#Работает с субъективными оценками, поэтому проверяет ее с помощью средних по матрице, по строке, по столбцу
#Не стоит минимизировать MSE - потому что нужны рекомендации только с большими значениями

data = Dataset.load_builtin('ml-100k')

#Создаем модель
algo = SVD(n_factors=160, 
           n_epochs=100, 
           lr_all=0.005, #скорость обучения, шаг модификации
           reg_all=0.1, #гамма регулирующая
           biased=True, 
           random_state=42 
          )
#Обучаем модель
cross_validate(algo, 
               data, 
               measures=['RMSE', 'MAE'], 
               cv=5, 
               verbose=True
              )

#проверяем модель на случайном пользователе
userid = str(196)
itemid = str(302)
actual_rating = 4
print(algo.predict(userid, 302, 4))